In [16]:
##importing all the required libraries

import cv2
import numpy as np
import torch
from PIL import Image
import matplotlib.pyplot as plt
import os # Import the os module



Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2025-3-22 Python-3.11.11 torch-2.6.0+cu124 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved detected image to: detected_images/detected_cat_1.jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:894: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Saved detected image to: detected_images/detected_cat_2.jpg


Saved detected image to: detected_images/detected_cat_3.jpg


In [ ]:
# Create a directory for detected images if it doesn't exist
detected_images_dir = 'detected_images'
if not os.path.exists(detected_images_dir):
    os.makedirs(detected_images_dir)




In [ ]:
# Using the YOLOv5 model for object detection and segmentation using kmeans
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

def process_image(img_path):
    # Object Detection
    # Check if the image file exists
    if not os.path.exists(img_path):
        print(f"Error: Image file not found: {img_path}")
        return None, None  # Return None if image not found

    img = cv2.imread(img_path)

    # Check if image loading was successful
    if img is None:
        print(f"Error: Could not read image file: {img_path}")
        return None, None  # Return None if image loading failed

    results = model(img)
    detections = results.pandas().xyxy[0]

    # Draw bounding boxes
    for _, row in detections.iterrows():
        if row['name'] == 'cat':
            cv2.rectangle(img, (int(row['xmin']), int(row['ymin'])),
                          (int(row['xmax']), int(row['ymax'])), (0,255,0), 2)

    # Image Segmentation (K-means clustering)
    pixel_values = img.reshape((-1, 3)).astype(np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    _, labels, centers = cv2.kmeans(pixel_values, 4, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    segmented = centers[np.uint8(labels.flatten())].reshape(img.shape)

    return img, segmented



In [ ]:
# Process three cat images and then they are segmented and detected and the stored in a separate directory named as detected_images
for i in range(1, 4):
    img_path = f'cat_{i}.jpg'
    detected, segmented = process_image(img_path)

    # Check if processing was successful
    if detected is not None and segmented is not None:
        plt.figure(figsize=(15,5))
        plt.subplot(1,2,1), plt.imshow(cv2.cvtColor(detected, cv2.COLOR_BGR2RGB))
        plt.title('Object Detection'), plt.axis('off')

        plt.subplot(1,2,2), plt.imshow(cv2.cvtColor(segmented, cv2.COLOR_BGR2RGB))
        plt.title('Image Segmentation'), plt.axis('off')
        plt.show()

        # Save the detected image to the directory
        detected_img_path = os.path.join(detected_images_dir, f'detected_cat_{i}.jpg')
        cv2.imwrite(detected_img_path, detected)
        print(f"Saved detected image to: {detected_img_path}")
    else:
        print(f"Skipping image {img_path} due to processing error.")